In [1]:
%load_ext autoreload
%autoreload 2

from helpers import *

from surprise import Dataset
from surprise import Reader
from ALS_implementation import ALS
from surprise.model_selection import GridSearchCV

path_dataset = "data/data_train.csv"
data = load_data(path_dataset)
dataset = Dataset.load_from_df(data[['user', 'movie', 'rating']], Reader(rating_scale=(1, 5)))

data.head() 

,user,movie,rating
0,r44,c1,4
1,r61,c1,3
2,r67,c1,4
3,r72,c1,3
4,r86,c1,5


![alt text](ALS.png "Title")

In [2]:
param_grid = {"n_epochs": [25], "num_features": [8,9], "lambda_all": [0.08,0.081,0.082]}
gs = GridSearchCV(ALS, param_grid, measures=['rmse', 'mae'], cv=2, n_jobs=-1, joblib_verbose=1000) 
gs.fit(dataset)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print(gs.cv_results)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:  9.4min remaining: 47.0min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:  9.6min remaining: 28.7min
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:  9.9min remaining: 19.8min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed: 10.2min remaining: 14.3min
[Parallel(n_jobs=-1)]: Done   6 out of  12 | elapsed: 10.5min remaining: 10.5min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed: 10.8min remaining:  7.7min
[Parallel(n_jobs=-1)]: Done   8 out of  12 | elapsed: 11.1min remaining:  5.6min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed: 11.3min remaining:  3.8min
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed: 11.6min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 12.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapse

In [3]:
param_grid = {"n_epochs": [30], "num_features": [8], "lambda_all": [0.082,0.083,0.084]}
gs = GridSearchCV(ALS, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1, joblib_verbose=1000) 
gs.fit(dataset)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed: 11.5min remaining: 40.3min
[Parallel(n_jobs=-1)]: Done   3 out of   9 | elapsed: 11.8min remaining: 23.6min
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed: 12.1min remaining: 15.1min
[Parallel(n_jobs=-1)]: Done   5 out of   9 | elapsed: 12.4min remaining:  9.9min
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed: 12.6min remaining:  6.3min
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed: 12.8min remaining:  3.7min
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 13.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 13.2min finished


In [4]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print(gs.cv_results)

0.9934012357606444
{'n_epochs': 30, 'num_features': 8, 'lambda_all': 0.084}
{'split0_test_rmse': array([0.9924798 , 0.99236831, 0.99226688]), 'split1_test_rmse': array([0.99443614, 0.99432018, 0.99421389]), 'split2_test_rmse': array([0.99390923, 0.99381125, 0.99372294]), 'mean_test_rmse': array([0.99360839, 0.99349992, 0.99340124]), 'std_test_rmse': array([0.00082651, 0.0008267 , 0.00082677]), 'rank_test_rmse': array([3, 2, 1], dtype=int64), 'split0_test_mae': array([0.80239087, 0.80250685, 0.80263009]), 'split1_test_mae': array([0.80401327, 0.80412062, 0.80423438]), 'split2_test_mae': array([0.80337997, 0.80350281, 0.80363208]), 'mean_test_mae': array([0.80326137, 0.80337676, 0.80349885]), 'std_test_mae': array([0.00066763, 0.00066482, 0.00066169]), 'rank_test_mae': array([1, 2, 3], dtype=int64), 'mean_fit_time': array([660.19548448, 666.71630351, 651.88606079]), 'std_fit_time': array([1.91962782, 2.2341046 , 6.94648341]), 'mean_test_time': array([3.57522607, 2.69926286, 2.12386417]),

In [5]:
param_grid = {"n_epochs": [30], "num_features": [8], "lambda_all": [0.084,0.085,0.086]}
gs = GridSearchCV(ALS, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1, joblib_verbose=1000) 
gs.fit(dataset)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print(gs.cv_results)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:  7.8min remaining: 15.6min
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  7.9min remaining:  7.9min
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:  8.2min remaining:  4.1min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  8.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  8.6min finished
1.0054997967173716
{'n_epochs': 30, 'num_features': 8, 'lambda_all': 0.086}
{'split0_test_rmse': array([1.00707842, 1.00677173, 1.00647623]), 'split1_test_rmse': array([1.00510211, 1.00480711, 1.00452336]), 'mean_test_rmse': array([1.00609026, 1.00578942, 1.0054998 ]), 'std_test_rmse': array([0.00098815, 0.00098231, 0.00097644]), 'rank_test_rmse': array([3, 2, 1], dtype=int64), 'split0_test_mae': array([0.81255878, 0.81253084, 0.81250989]), 'split1_test_mae': ar

In [7]:
param_grid = {"n_epochs": [30], "num_features": [8], "lambda_all": [0.087,0.088,0.089]}
gs = GridSearchCV(ALS, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1, joblib_verbose=1000) 
gs.fit(dataset)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print(gs.cv_results)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed: 11.7min remaining: 40.8min
[Parallel(n_jobs=-1)]: Done   3 out of   9 | elapsed: 12.0min remaining: 24.1min
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed: 12.3min remaining: 15.4min
[Parallel(n_jobs=-1)]: Done   5 out of   9 | elapsed: 12.5min remaining: 10.0min
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed: 12.7min remaining:  6.4min
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed: 13.1min remaining:  3.7min
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 13.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 13.4min finished
0.9931152870535876
{'n_epochs': 30, 'num_features': 8, 'lambda_all': 0.089}
{'split0_test_rmse': array([0.9929362 , 0.99288229, 0.99283755]), 'split1_test_rmse': array([0.99292643, 0.99285521, 0.99279377]), 'split2_test_rms

In [8]:
param_grid = {"n_epochs": [30], "num_features": [8], "lambda_all": [0.09,0.093,0.096]}
gs = GridSearchCV(ALS, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1, joblib_verbose=1000) 
gs.fit(dataset)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print(gs.cv_results)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed: 12.2min remaining: 42.8min
[Parallel(n_jobs=-1)]: Done   3 out of   9 | elapsed: 12.4min remaining: 24.9min
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed: 12.8min remaining: 16.0min
[Parallel(n_jobs=-1)]: Done   5 out of   9 | elapsed: 13.0min remaining: 10.4min
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed: 13.3min remaining:  6.7min
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed: 13.5min remaining:  3.9min
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 13.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 13.9min finished
0.9927309198002625
{'n_epochs': 30, 'num_features': 8, 'lambda_all': 0.093}
{'split0_test_rmse': array([0.9914656 , 0.99141051, 0.99143592]), 'split1_test_rmse': array([0.99224211, 0.99217308, 0.99218306]), 'split2_test_rms

In [ ]:
param_grid = {"n_epochs": [30], "num_features": [8], "lambda_all": [0.092,0.093,0.094]}
gs = GridSearchCV(ALS, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1, joblib_verbose=1000) 
gs.fit(dataset)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print(gs.cv_results)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed: 12.1min remaining: 42.5min
[Parallel(n_jobs=-1)]: Done   3 out of   9 | elapsed: 12.3min remaining: 24.7min
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed: 12.7min remaining: 15.8min
[Parallel(n_jobs=-1)]: Done   5 out of   9 | elapsed: 12.8min remaining: 10.2min
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed: 13.2min remaining:  6.6min
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed: 13.4min remaining:  3.8min


In [6]:
submission = pd.read_csv("data/sample_submission.csv")
trainset = dataset.build_full_trainset()
gsbf = gs.best_estimator['rmse'].fit(trainset)

number of items in preprocess_trainset_data: 1000, number of users: 10000

start the ALS algorithm...
RMSE on training set: 1.0279557692829417.
RMSE on training set: 1.002643700880776.
RMSE on training set: 0.984104400950518.
RMSE on training set: 0.9725554764695716.
RMSE on training set: 0.9649053241359405.
RMSE on training set: 0.9593962705374679.
RMSE on training set: 0.9554039026823504.
RMSE on training set: 0.952536228342509.
RMSE on training set: 0.9504453053662352.
RMSE on training set: 0.948866722777238.
RMSE on training set: 0.9476306025435011.
RMSE on training set: 0.946633145835352.
RMSE on training set: 0.9458099253295328.
RMSE on training set: 0.945119423907407.
RMSE on training set: 0.9445336891663806.
RMSE on training set: 0.9440329854033788.
RMSE on training set: 0.9436026810899607.
RMSE on training set: 0.9432314284306135.
RMSE on training set: 0.9429101013482459.
RMSE on training set: 0.9426311748098132.
RMSE on training set: 0.9423883539770682.
RMSE on training set: 

In [7]:
def predict(user, movie):
    return int(round(recommenders[cluster_dict[movie]].predict(user, movie).est))

In [8]:
submission['Prediction'] = [int(round(gsbf.predict(user, movie).est)) for [user, movie] in submission['Id'].str.split('_')]
submission.to_csv(r'data/submission_als_adri_1.csv', index=False)